In [1]:
import pandas as pd 
from sklearn.neighbors import DistanceMetric
import numpy as np


import cx_Oracle as cx

import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import mariadb
import sys
import datetime

from scipy.spatial import cKDTree
from shapely.geometry import Point
import geopandas as gpd
import re

In [2]:
df_arcep=pd.read_excel(".\\Livrable de référence ARCEP2023_Data_Agglos_v3_Intermediaire2_FRE.xlsx",usecols=['Date','AutoCall Scenario Name','[Down] APP Throughput [Mbps]','Usage','Serving PCI','Serving Cell EARFCN','Address','Call Start Lat','Call_Start_Lon','LTE CA Network Ratio','Serving Cell RSRP','PCI_5G','Pcell_PCI','NR-ARFCN','Pcell_NR-ARFCN'])

In [3]:
df2=pd.read_csv('.\\liste_sites_atoll.csv',encoding="ISO-8859-1",delimiter=";")

In [4]:
import cx_Oracle as cx
conora = cx.connect('fhattab'+'/'+'atoll'+'@//192.168.105.23:1521/orclpdb',encoding='iso-8859-1')
cursor = conora.cursor()

df_atoll = pd.read_sql("select * from ATOLL_5GMRAT.sites where (FIELDCHAR2 ='INT' or FIELDCHAR2 ='DTX' or FIELDCHAR2 ='FTX' \
    or FIELDCHAR2 ='SRI')",conora)


for i,j in zip(df_atoll,df2):
    df_atoll.rename(columns={i:j},inplace=True) 
usecols=["Nom","Longitude","Latitude","ETAT SITE - Config"]
df_atoll=df_atoll[usecols]

In [5]:
df_export=pd.read_csv('.\\export_Cellules_5g_20230725.csv',delimiter=",")

In [6]:
df_arcep=df_arcep[df_arcep["Usage"]!='incar']
df_arcep=df_arcep[df_arcep["AutoCall Scenario Name"].str.startswith(("DL_250","UL_50"))]
df_arcep.head()

,Call_Start_Lon,Call Start Lat,AutoCall Scenario Name,LTE CA Network Ratio,Serving Cell RSRP,Serving Cell EARFCN,Serving PCI,PCI_5G,NR-ARFCN,Pcell_NR-ARFCN,Pcell_PCI,[Down] APP Throughput [Mbps],Usage,Address,Date
43,0.22640,47.99675,DL_250MO_IPV6 (App_HTTP),0.0,-107.4,375,147,651,645024,NaN,NaN,40741.777,indoor,284 avenue Bollée 72000 Le Mans,02/05/2023
44,0.22714,47.99656,UL_50MO_IPV6 (App_HTTP),0.0,-106.6,375,147,651,645024,NaN,NaN,NaN,indoor,284 avenue Bollée 72000 Le Mans,02/05/2023
58,0.22714,47.99656,DL_250MO_IPV6 (App_HTTP),0.0,-110.6,375,483,760;651;760,645024,NaN,NaN,1794.540,outdoor,284 avenue Bollée 72000 Le Mans,02/05/2023
59,0.22714,47.99656,UL_50MO_IPV6 (App_HTTP),0.0,-110.5,375,483,760,645024,NaN,NaN,NaN,outdoor,284 avenue Bollée 72000 Le Mans,02/05/2023
83,0.19663,48.00681,DL_250MO_IPV6 (App_HTTP),0.0,-95.3,3350,435,939,645024,NaN,NaN,75022.914,indoor,5 rue du Hallai 72000 Le Mans,02/05/2023


In [8]:
df_point=df_arcep.copy()

In [9]:
df_arcep=df_point.copy()

In [10]:
df_export=pd.read_csv('.\\export_Lncel_20230724 (2).csv',delimiter=",")

In [11]:
df_export['site']

KeyError: 'site'

In [12]:
df_export=pd.read_csv('.\\export_Lncel_20230724 (2).csv',delimiter=",")
df_zb=pd.read_csv('.\\zone_blache_2023_07_24.csv',delimiter=";",usecols=["site","cell_name","earfcn_dl","PCI"])
df_zb.columns=df_export[["PCI","earfcnDL","site","name"]].columns
df_glb=pd.concat([df_export[["PCI","earfcnDL","site","name"]],df_zb]).reset_index()
df_export.shape

KeyError: "['site'] not in index"

In [ ]:
class Arcep_point():
    def __init__(self,df,df_export):
        self.df=df
        self.df_export=df_export
    def merge_pci(self):
        self.df.loc[self.df['PCI_5G'].isnull(),'PCI_5G']=self.df["Pcell_PCI"]
        self.df.loc[self.df['NR-ARFCN'].isnull(),'NR-ARFCN']=self.df["Pcell_NR-ARFCN"]
        return None
    def pre_traitement (self):
        self.df['PCI_5G']=self.df['PCI_5G'].apply(lambda x: str(x).replace("->",";"))

        self.df['NR-ARFCN']=self.df['NR-ARFCN'].apply(lambda x: str(x).replace("->",";"))
        
        self.df['NR-ARFCN']=self.df['NR-ARFCN'].apply(lambda x: str(x).split(';'))
        self.df['PCI_5G']=self.df['PCI_5G'].apply(lambda x: str(x).split(';'))
        self.df_explode=self.df.explode("PCI_5G").explode("NR-ARFCN")
        self.df_explode["NR-ARFCN"]=self.df_explode["NR-ARFCN"].str[0:3]+"000"
        self.df_explode.loc[self.df_explode["NR-ARFCN"]=="156000","NR-ARFCN"]="0"
        print(self.df_explode.head())
        return None
        
    def oss_export_5G(self):
        self.df_export["site"]=self.df_export["CellName"].str[:9]
        self.df_export=df_export[self.df_export["CellName"].notnull()]
        self.df_ref=pd.merge(self.df_export,df_atoll,how="left",left_on="site",right_on="Nom")
        return None
    def oss_export_4G(self,df_zb):
        self.df_glb=pd.concat([df_export[["PCI","earfcnDL","site","name"]],df_zb]).reset_index()
        self.df_ref=pd.merge(self.df_glb,df_atoll,how="left",left_on="site",right_on="Nom")
        
        return None
    def calculate_site(self):
        print(self.df_explode.head())
        gpd1 = gpd.GeoDataFrame(self.df_explode, geometry=gpd.points_from_xy(self.df_explode['Call_Start_Lon'], self.df_explode['Call Start Lat'])) 
        gpd2 = gpd.GeoDataFrame(self.df_ref, geometry=gpd.points_from_xy(self.df_ref.Longitude, self.df_ref.Latitude))
        gpd2=gpd2[(gpd2["Longitude"].notnull()) & (gpd2["Latitude"].notnull())] 
        gpd1=gpd1[gpd1["NR-ARFCN"].isin(["645000","643334","2063331","0","155600","650000"])] 


        gpd1=gpd1[gpd1["PCI_5G"]!="nan"]
        gpd1=gpd1[(gpd1['Call_Start_Lon'].notnull()) & (gpd1['Call Start Lat'].notnull())&(gpd1['Call_Start_Lon'].notnull()!=0)&(gpd1['Call Start Lat'].notnull()!=0)]
        gpd1=gpd1[gpd1["PCI_5G"].notnull()]
        print(gpd1.head())
        dist,idx=np.empty((0, 2)),np.empty((0, 2))
        df_b=pd.DataFrame()
        for i in range(0,len(gpd1["PCI_5G"])):    
            print(i)
            
            try:
                nA = np.array(list(pd.DataFrame(gpd1.iloc[i].to_frame().T).geometry.apply(lambda x: (x.x, x.y))))
                print(gpd1.iloc[i]["PCI_5G"])
                print(gpd1.iloc[i]["NR-ARFCN"])
                nB = np.array(list(pd.DataFrame(gpd2[(gpd2["physCellId"]==int(pd.DataFrame(gpd1.iloc[i].to_frame().T)["PCI_5G"]))&((gpd2["nrarfcn"]==int(pd.DataFrame(gpd1.iloc[i].to_frame().T)["NR-ARFCN"])))].reset_index()).geometry.apply(lambda x: (x.x, x.y))))
                gpd_exp=gpd2[(gpd2["physCellId"]==int(pd.DataFrame(gpd1.iloc[i].to_frame().T)["PCI_5G"]))&(gpd2["nrarfcn"]==int(pd.DataFrame(gpd1.iloc[i].to_frame().T)["NR-ARFCN"]))].reset_index()
                nB = np.array(list(pd.DataFrame(gpd2[(gpd2["physCellId"]==int(pd.DataFrame(gpd1.iloc[i].to_frame().T)["PCI_5G"]))&((gpd2["nrarfcn"]==int(pd.DataFrame(gpd1.iloc[i].to_frame().T)["NR-ARFCN"])))].reset_index()).geometry.apply(lambda x: (x.x, x.y))))
                btree = cKDTree(nB)
                dist_i, idx_i =btree.query(nA,k=2)
                print(dist_i)   
                dist=np.concatenate([dist,dist_i])
                idx=np.concatenate([idx,idx_i])
                df_b=df_b.append(pd.concat([pd.concat([gpd1.iloc[i].to_frame().T]*2).reset_index(),gpd_exp.iloc[idx_i.tolist()[0],:].drop(columns="geometry").reset_index(drop=True),pd.Series(dist_i[0],name='dist')],axis=1))
                print(df_append.head())
            except:
                pass
        return df_b

In [ ]:
obj_5G=Arcep_point(df_arcep,df_export)
obj_5G.merge_pci()
obj_5G.oss_export_5G()
obj_5G.pre_traitement()

In [ ]:
obj_5G.merge_pci()

In [ ]:
obj_5G.oss_export_5G()

In [ ]:
obj_5G.pre_traitement()

In [ ]:
obj_5G.df_explode

In [ ]:
df_5G=obj_5G.calculate_site()

In [ ]:
df_5G.shape

In [ ]:
def haversine_vectorize(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2=lon1.astype("float"),lat1.astype("float"),lon2.astype("float"),lat2.astype("float")
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    newlon = lon2 - lon1
    newlat = lat2 - lat1
        
    haver_formula = np.sin(newlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(newlon/2.0)**2
    dist = 2 * np.arcsin(np.sqrt(haver_formula ))
    km = 6367 * dist #6367 for distance in KM for miles use 3958
    return km

In [ ]:
 df_5G["distance_Km"]=haversine_vectorize(df_5G["Call_Start_Lon"],df_5G['Call Start Lat'],df_5G["Longitude"],df_5G["Latitude"])

In [ ]:
df_5G["dist_min"]=df_5G.groupby(['PCI_5G','NR-ARFCN','Call_Start_Lon','Call Start Lat','Usage'])["distance_Km"].transform('min')

In [ ]:
df_5G=df_5G[df_5G["distance_Km"]==df_5G['dist_min']]

In [ ]:
df_c=df_5G.drop_duplicates(subset=["Address","PCI_5G","NR-ARFCN","Usage"], keep='first')


In [ ]:

import folium
import seaborn as sns


mai=folium.Map(location=[48.862, 2.346], zoom_start=6)



for grp_name,df_grp in df_c.groupby('Usage'):
    feature_group=folium.FeatureGroup(grp_name)
    locations = df_grp[['Call Start Lat','Call_Start_Lon']]
    locationlist = locations.values.tolist() 
    locations_nigh=df_grp[['Latitude', 'Longitude']]
    location_nigh = locations_nigh.values.tolist()
    
    for point,pt in zip(range(0, len(locationlist)),range(0, len(location_nigh))):
        liste=[]
        liste.append(tuple(locationlist[point]))
        liste.append(tuple(location_nigh[pt]))
        
        
        iframe = folium.IFrame('Addresse:' + str(df_grp.iloc[point,:]["Address"]) + '<br>' + 'PCI: '+str(df_grp.iloc[point,:]["PCI_5G"]) + '<br>' + 'USage: ' + str(df_grp.iloc[point,:]["Usage"]))
        iframe_pt = folium.IFrame('Cell_name:' + str(df_grp.iloc[pt,:]["CellName"]) + '<br>' + 'PCI: '+str(df_grp.iloc[pt,:]["PCI_5G"])+ '<br>' + 'CA  Ratio: '+str(df_grp.iloc[pt,:]['LTE CA Network Ratio'])+'<br>' + 'Serving Cell RSRP: '+str(df_grp.iloc[pt,:]['Serving Cell RSRP']))
     
        popup = folium.Popup(iframe, min_width=200, max_width=300)
        
        color_pallete = sns.color_palette()
        color_pallete = sns.color_palette("Set2", 8000)
        color_pallete = color_pallete.as_hex()

        folium.Marker(locationlist[point],
                  popup=popup).add_to(feature_group)
        feature_group.add_to(mai)
        folium.PolyLine(liste,color="red", weight=2.5, opacity=3).add_to(feature_group)
        feature_group.add_to(mai)
                          
        popup_pt = folium.Popup(iframe_pt, min_width=200, max_width=250)
        folium.CircleMarker(location_nigh[pt],popup=popup_pt,).add_to(feature_group)
        feature_group.add_to(mai)

folium.LayerControl().add_to(mai)
    
mai.save('./Arcep_map_Class_5G.html')
mai

In [ ]:
df_arcep=df_point.copy()

In [ ]:
  
class Arcep_point():
    def __init__(self,df,df_export,tech):
        self.df=df
        self.tech=tech
        self.df_export=df_export
        if tech=="5G":
            self.pci='PCI_5G'
            self.arfcn='NR-ARFCN'
        elif tech=="4G":
            self.pci='Serving PCI'
            self.arfcn='Serving Cell EARFCN'      
    
    def merge_pci(self):
        self.df.loc[self.df[self.pci].isnull(),self.pci]=self.df["Pcell_PCI"]
        self.df.loc[self.df[self.arfcn].isnull(),self.arfcn]=self.df["Pcell_NR-ARFCN"]
        return None
    def pre_traitement (self):
        self.df[self.pci]=self.df[self.pci].apply(lambda x: str(x).replace("->",";"))

        self.df[self.arfcn]=self.df[self.arfcn].apply(lambda x: str(x).replace("->",";"))
        
        self.df[self.arfcn]=self.df[self.arfcn].apply(lambda x: str(x).split(';'))
        self.df[self.pci]=self.df[self.pci].apply(lambda x: str(x).split(';'))
        self.df_explode=self.df.explode(self.pci).explode(self.arfcn)
        return None
        
    def oss_export_5G(self):
        
        self.df_explode[self.arfcn]=self.df_explode[self.arfcn].str[0:3]+"000"
        self.df_explode.loc[self.df_explode[self.arfcn]=="156000",self.arfcn]="0"
        self.df_export["site"]=self.df_export["CellName"].str[:9]
        self.df_export=self.df_export[self.df_export["CellName"].notnull()]
        self.df_ref=pd.merge(self.df_export,df_atoll,how="left",left_on="site",right_on="Nom")
        return None
    def oss_export_4G(self,df_zb):
        self.df_export["site"]=self.df_export["name"].str[:9]
        df_zb.columns=self.df_export[["PCI","earfcnDL","site","name"]].columns
        self.df_glb=pd.concat([self.df_export[["PCI","earfcnDL","site","name"]],df_zb]).reset_index()
        self.df_ref=pd.merge(self.df_glb,df_atoll,how="left",left_on="site",right_on="Nom")
        
        return None
    def Gpd_creating(self):
        gpd1 = gpd.GeoDataFrame(self.df_explode, geometry=gpd.points_from_xy(self.df_explode['Call_Start_Lon'], self.df_explode['Call Start Lat'])) 
        gpd2 = gpd.GeoDataFrame(self.df_ref, geometry=gpd.points_from_xy(self.df_ref.Longitude, self.df_ref.Latitude))
        gpd2=gpd2[(gpd2["Longitude"].notnull()) & (gpd2["Latitude"].notnull())] 
        arfcn_list=[]
        print(gpd1.head())
        
        if self.tech=="5G":
            arfcn_list=["645000","643334","2063331","0","155600","650000"]
            PCI_oss="physCellId"
            arfcn_oss="nrarfcn"
            
        elif self.tech=="4G":
            arfcn_list=["9460","3350","1675","9335","6275","351","375","6200","400"]           
            PCI_oss="PCI"
            arfcn_oss="earfcnDL"
        print(arfcn_list)
        print(self.arfcn)
        gpd1=gpd1[gpd1[self.arfcn].isin(arfcn_list)] 
        
    

        gpd1=gpd1[gpd1[self.pci]!="nan"]
        gpd1=gpd1[(gpd1['Call_Start_Lon'].notnull()) & (gpd1['Call Start Lat'].notnull())&(gpd1['Call_Start_Lon'].notnull()!=0)&(gpd1['Call Start Lat'].notnull()!=0)]
        gpd1=gpd1[gpd1[self.pci].notnull()]
        print(gpd1.head())
        dist,idx=np.empty((0, 2)),np.empty((0, 2))
        df_b=pd.DataFrame()
        for i in range(0,len(gpd1[self.pci])):  
            try:
                nA = np.array(list(pd.DataFrame(gpd1.iloc[i].to_frame().T).geometry.apply(lambda x: (x.x, x.y))))
               
                nB = np.array(list(pd.DataFrame(gpd2[(gpd2[PCI_oss]==int(pd.DataFrame(gpd1.iloc[i].to_frame().T)[self.pci]))&((gpd2[arfcn_oss]==int(pd.DataFrame(gpd1.iloc[i].to_frame().T)[self.arfcn])))].reset_index()).geometry.apply(lambda x: (x.x, x.y))))
                gpd_exp=gpd2[(gpd2[PCI_oss]==int(pd.DataFrame(gpd1.iloc[i].to_frame().T)[self.pci]))&(gpd2[arfcn_oss]==int(pd.DataFrame(gpd1.iloc[i].to_frame().T)[self.arfcn]))].reset_index()
                
                nB = np.array(list(pd.DataFrame(gpd2[(gpd2[PCI_oss]==int(pd.DataFrame(gpd1.iloc[i].to_frame().T)[self.pci]))&((gpd2[arfcn_oss]==int(pd.DataFrame(gpd1.iloc[i].to_frame().T)[self.arfcn])))].reset_index()).geometry.apply(lambda x: (x.x, x.y))))
                print(nA)
                print(nB)
                btree = cKDTree(nB)
                dist_i, idx_i =btree.query(nA,k=2)
                print(dist_i)   
                dist=np.concatenate([dist,dist_i])
                idx=np.concatenate([idx,idx_i])
                df_b=df_b.append(pd.concat([pd.concat([gpd1.iloc[i].to_frame().T]*2).reset_index(),gpd_exp.iloc[idx_i.tolist()[0],:].drop(columns="geometry").reset_index(drop=True),pd.Series(dist_i[0],name='dist')],axis=1))
                print(df_b.head())
            except:
                pass

          

        return df_b

In [ ]:
obj_5G=Arcep_point(df_arcep,df_export,"5G")


In [ ]:
obj_5G.merge_pci()

In [ ]:
obj_5G.pre_traitement()

In [ ]:
obj_5G.oss_export_5G()


In [ ]:
df_nr=obj_5G.Gpd_creating()

In [ ]:
df_nr.head()

In [ ]:
df_export_4G=pd.read_csv('.\\export_Lncel_20230724 (2).csv',delimiter=",")

In [ ]:
obj_4G=Arcep_point(df_arcep,df_export_4G,"4G")


In [ ]:
df_zb=pd.read_csv('.\\zone_blache_2023_07_24.csv',delimiter=";",usecols=["site","cell_name","earfcn_dl","PCI"])



In [ ]:
obj_4G.pre_traitement()

In [ ]:
obj_4G.oss_export_4G(df_zb)

In [ ]:
df_4G=obj_4G.Gpd_creating()

In [ ]:
df_4G.head()

In [ ]:
def haversine_vectorize(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2=lon1.astype("float"),lat1.astype("float"),lon2.astype("float"),lat2.astype("float")
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    newlon = lon2 - lon1
    newlat = lat2 - lat1
        
    haver_formula = np.sin(newlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(newlon/2.0)**2
    dist = 2 * np.arcsin(np.sqrt(haver_formula ))
    km = 6367 * dist #6367 for distance in KM for miles use 3958
    return km

In [ ]:
 df_4G["distance_Km"]=haversine_vectorize(df_4G["Call_Start_Lon"],df_4G['Call Start Lat'],df_4G["Longitude"],df_4G["Latitude"])

In [ ]:
df_4G["dist_min"]=df_4G.groupby(['Serving PCI','Serving Cell EARFCN','Call_Start_Lon','Call Start Lat','Usage'])["distance_Km"].transform('min')

In [ ]:
df_4G=df_4G[df_4G["distance_Km"]==df_4G['dist_min']]

In [ ]:
df_c=df_5G.drop_duplicates(subset=["Address","Pcell_PCI","Serving Cell EARFCN","Usage"], keep='first')


In [ ]:
import folium
import seaborn as sns


mai=folium.Map(location=[48.862, 2.346], zoom_start=6)



for grp_name,df_grp in df_c.groupby('Usage'):
    feature_group=folium.FeatureGroup(grp_name)
    locations = df_grp[['Call Start Lat','Call_Start_Lon']]
    locationlist = locations.values.tolist() 
    locations_nigh=df_grp[['Latitude', 'Longitude']]
    location_nigh = locations_nigh.values.tolist()
    
    for point,pt in zip(range(0, len(locationlist)),range(0, len(location_nigh))):
        liste=[]
        liste.append(tuple(locationlist[point]))
        liste.append(tuple(location_nigh[pt]))
        
        
        iframe = folium.IFrame('Addresse:' + str(df_grp.iloc[point,:]["Address"]) + '<br>' + 'PCI: '+str(df_grp.iloc[point,:]["PCI_5G"]) + '<br>' + 'USage: ' + str(df_grp.iloc[point,:]["Usage"]))
        iframe_pt = folium.IFrame('Cell_name:' + str(df_grp.iloc[pt,:]["CellName"]) + '<br>' + 'PCI: '+str(df_grp.iloc[pt,:]["PCI_5G"])+ '<br>' + 'CA  Ratio: '+str(df_grp.iloc[pt,:]['LTE CA Network Ratio'])+'<br>' + 'Serving Cell RSRP: '+str(df_grp.iloc[pt,:]['Serving Cell RSRP']))
     
        popup = folium.Popup(iframe, min_width=200, max_width=300)
        
        color_pallete = sns.color_palette()
        color_pallete = sns.color_palette("Set2", 8000)
        color_pallete = color_pallete.as_hex()

        folium.Marker(locationlist[point],
                  popup=popup).add_to(feature_group)
        feature_group.add_to(mai)
        folium.PolyLine(liste,color="red", weight=2.5, opacity=3).add_to(feature_group)
        feature_group.add_to(mai)
                          
        popup_pt = folium.Popup(iframe_pt, min_width=200, max_width=250)
        folium.CircleMarker(location_nigh[pt],popup=popup_pt,).add_to(feature_group)
        feature_group.add_to(mai)

folium.LayerControl().add_to(mai)
    
mai.save('./Arcep_map_Class_5G.html')
mai